### Extract Aspect from three qualitative question from GenEd

In [1]:
pip install nltk

     |████████████████████████████████| 1.4MB 20.4MB/s eta 0:00:01
     |████████████████████████████████| 307kB 7.4MB/s eta 0:00:01
     |████████████████████████████████| 675kB 3.9MB/s eta 0:00:01
     |████████████████████████████████| 81kB 3.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import ast
from nltk import word_tokenize, pos_tag, pos_tag_sents

In [3]:
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords


from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet 
# Create WordNetLemmatizer object 


lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
df = pd.read_csv('Cleaned_CES_Pre_1st.csv')
second_df = pd.read_csv('Cleaned_CES_Final_1st.csv')
df = df.replace(np.nan, 'Neutral', regex=True) ### Some values are missing, use "Neutral" for missing values
df = df.drop(df.columns[[0]],axis=1) ### Drop extra index
test_df = df.iloc[0:, 0:] # Just for renaming purposes so that df variable can be used for testing
frame = [df,second_df]
test_df = pd.concat(frame)
test_df

,SUBJECT:,SECTION:,COURSE SCHEDULE,I can easily log-in and log-out my Canvas account.,I can access my course using the mobile Canvas app.,I can access my Canvas account using the available computers in JRU.,I have no issues accessing my Canvas account using JRU's wifi connection.,I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories.,I can easily get help when I have problems with my Canvas account.,I can access my Canvas account outside JRU.,...,Canvas has made it easy for me to: Share ideas on group discussions,"I can answer the assessment materials (quiz, exam) posted in Canvas.",I find the course materials on Canvas aligned with the: Quizzes,"I find the course materials on Canvas aligned with the: Major (Prelim, Mideterm, Final) Exam",I find the course materials on Canvas aligned with the: Other tasks,I have been give enough time to accomplish the activities on Canvas.,What I like best in Canvas is/are:,What I like least in Canvas is/are:,I would like to suggest:,Unnamed: 0
0,HIS C101- Readings in Philippine History,BSA 101A,12:00 PM,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Neutral,Neutral,NaN
1,HIS C101- Readings in Philippine History,103R,12:18 PM,Disagree,Disagree,Disagree,Agree,Disagree,Agree,Agree,...,Disagree,Agree,Agree,Agree,Agree,Disagree,Neutral,Neutral,Neutral,NaN
2,HIS C101- Readings in Philippine History,101K,01:30 PM,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Disagree,Agree,Agree,Strongly Agree,Agree,Strongly Agree,I can easily answer my quizzes and other forum...,Some announcement are not showing up in my not...,Create a power point moduel not a word and sum...,NaN
3,HIS C101- Readings in Philippine History,102-F,12:00 PM,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Agree,Agree,Agree,Agree,Agree,Agree,It can di at home,Some of the instructions are not fully underst...,To be more active at face to face and lessen t...,NaN
4,HIS C101- Readings in Philippine History,101M,10:30 AM,Strongly Agree,Agree,Agree,Agree,Agree,Agree,Agree,...,Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,give me some extra grades in specific course.,Neutral,Neutral,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,HIS C101- Readings in Philippine History,101LM,12:00 PM,Agree,Disagree,Agree,Agree,Agree,Agree,Agree,...,Disagree,Agree,Agree,Agree,Agree,Agree,NaN,NaN,NaN,2035.0
532,HIS C101- Readings in Philippine History,101 legal management,12:00 PM,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,NaN,NaN,NaN,2037.0
533,HIS C101- Readings in Philippine History,BSTM 1-1,9:00 AM,Strongly Agree,Strongly Agree,Agree,Agree,Agree,Agree,Strongly Agree,...,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Agree,NaN,NaN,NaN,2075.0
534,HIS C101- Readings in Philippine History,101 P,9:00 AM,Strongly Agree,Strongly Agree,Agree,Agree,Agree,Agree,Agree,...,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,NaN,NaN,NaN,2082.0


In [5]:
test_df = test_df.iloc[0:, -4:-1]
test_df

,What I like best in Canvas is/are:,What I like least in Canvas is/are:,I would like to suggest:
0,Neutral,Neutral,Neutral
1,Neutral,Neutral,Neutral
2,I can easily answer my quizzes and other forum...,Some announcement are not showing up in my not...,Create a power point moduel not a word and sum...
3,It can di at home,Some of the instructions are not fully underst...,To be more active at face to face and lessen t...
4,give me some extra grades in specific course.,Neutral,Neutral
...,...,...,...
531,NaN,NaN,NaN
532,NaN,NaN,NaN
533,NaN,NaN,NaN
534,NaN,NaN,NaN


In [6]:
stoplist = stopwords.words('english') + ['though']
def remove_stopWords(w): 
    w = ' '.join(word for word in w.split() if word not in stoplist)
    return w

In [7]:
## Reduce list dimension

def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [36]:
col_range = len(test_df.columns) # number of columns
test_df = test_df.replace(r'^\s*$', "neutral", regex=True) ## If row value is null, replace with neutral string
aspect_list = []
aspected = []
for i in range(0,3):
    col = test_df.columns[i] # The current column
    test_df.loc[:,col] = test_df[col].apply(lambda x : str.lower(str(x))) ## To Lower Case
    test_df.loc[:,col] = test_df[col].apply(lambda x : " ".join(re.findall('[\w]+',x))) # Remove Punctuations
    test_df.loc[:,col] = test_df[col].apply(lambda x : remove_stopWords(x)) # Remove Stop words
    
    ##POS TAGGING
    texts = test_df.loc[:,col].tolist()
    tagged_texts = pos_tag_sents(map(word_tokenize, texts)) ### Tag every word in a row with POS
    
    #inputTupples=ast.literal_eval(tagged_texts)
    flattened_tagged = flatten_list(tagged_texts)
    prevWord=''
    prevTag=''
    currWord=''
    aspectList=[]
    outputDict={}
    
    #Extracting Aspects
    
    for word,tag in flattened_tagged:
        if(tag=='NN' or tag=='NNP'):
            if(prevTag=='NN' or prevTag=='NNP'):
                currWord= prevWord + ' ' + word
                
            else:
                aspectList.append(prevWord.upper())
                currWord= word

        prevWord=currWord
        prevTag=tag
        
    aspected.append(aspectList)
    #Eliminating aspect which has 1 or less count
    for aspect in aspectList:
            if(aspectList.count(aspect)>1):
                    if(outputDict.keys()!=aspect):
                            outputDict[aspect]=aspectList.count(aspect)
    outputAspect=sorted(outputDict.items(), key=lambda x: x[1],reverse = True)
    aspect_list.append(outputAspect)
    #print(outputAspect)
    

In [38]:
len(aspected[0])

1082

In [15]:
df_cols = test_df.columns.tolist()

for i in range(0,col_range):
    print(i+1," ",df_cols[i])
    print(outputAspect, "\n")
    


1   What I like best in Canvas is/are:
[('CANVAS', 35), ('TIME', 24), ('NOTHING', 10), ('NONE', 9), ('TIME LIMIT', 7), ('FACE FACE', 6), ('PLEASE', 6), ('USE', 6), ('WAY', 6), ('NAN', 6), ('DATE', 5), ('STUDENT', 5), ('TASK', 5), ('QUIZ', 5), ('APP', 4), ('MODULE', 4), ('SUGGESTION', 4), ('ACCESS CODE', 4), ('PRESENTATION', 4), ('ACCESS', 4), ('CODE', 4), ('NAN NAN', 4), ('PASS', 3), ('NOTIFICATION', 3), ('CONTINUE', 3), ('CANVAS APP', 3), ('INFORMATION', 3), ('LIST', 3), ('IMPROVEMENT', 3), ('INTERNET', 3), ('WIFI', 3), ('INTERNET CONNECTION', 3), ('CLASSROOM', 3), ('CALENDAR', 3), ('NONE NAN', 3), ('NAN NAN NAN NAN NAN', 3), ('CANVAS NAN', 3), ('PACKAGE', 2), ('LOT', 2), ('POINT', 2), ('MAKE', 2), ('FIX PROBLEM', 2), ('SUGGEST PROFESSOR', 2), ('PASSCODE', 2), ('SUBJECT', 2), ('ACCESS CANVAS', 2), ('READ', 2), ('PUT', 2), ('CAMPUS', 2), ('KIND', 2), ('WEEK', 2), ('MATTER', 2), ('POST', 2), ('DEADLINE', 2), ('HASSLE', 2), ('MATERIAL', 2), ('LESSON', 2), ('PROFESSOR DISCUSS', 2), ('INST

In [13]:
test_df

,What I like best in Canvas is/are:,What I like least in Canvas is/are:,I would like to suggest:
0,neutral,neutral,neutral
1,neutral,neutral,neutral
2,easily answer quizzes forum need answer,announcement showing notification,create power point moduel word summary lesson ...
3,di home,instructions fully understandable,active face face lessen works canvas essay ans...
4,give extra grades specific course,neutral,neutral
...,...,...,...
531,nan,nan,nan
532,nan,nan,nan
533,nan,nan,nan
534,nan,nan,nan
